# Steering Vector
In order to delay our elements in the frequency domain, we will need to create a steering vector: An array which has a phase-shift for each element, depening on angle-of-incidence and frequency.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import IntSlider
# import pylab as plt

%matplotlib widget

# Generate the array geometry (for simplicity, a linear array)
PI = np.pi
Fs = 44100
speed_of_sound = 343.0

# Construct the 'probe' vector for the given angle
def steering_vector(f0, angle_deg, num_elements = 4, distance = 0.042):
    # Create an array with our microphone positions, in meters
    mic_positions = np.array([n*distance for n in np.arange(num_elements)])

    # Angle to radians
    angle_rad = np.radians(angle_deg)
    
    # Wavelength
    l = speed_of_sound / f0
    # mic_position in wavenumber
    d = mic_positions / l
    steering_vector = np.exp(-1j * 2*PI * d*np.sin(angle_rad))
    return steering_vector
    

Lets test our steering vector: We generate a few steering vectors and print the phase of each element for this angle (shown in degrees!), then the total gain (sum) and the summed phase we could get.
For a wavelength equal to our element spacing we expect additional side-lobes at 90 degree angles: For this angle-of-incidence we have a full 360 degrees phase shift and thus constructive interference. For some angle in the middle we would expect zero sensitivity where the phase between each consecutive element is 180 degrees.

In [2]:
np.set_printoptions(precision=2, suppress=True)

# set the wavelength of the incoming signal to the distance of our microphones
f0 = speed_of_sound / 0.042

# and plot out angle, angles of steering vector, sum of the elements (sensitivity) and sum of the phase shift (total phase shift of received signal)
print(f'{f0 = }')
for angle in [0, 30, 45, 60, 90]:
    sv = steering_vector(f0, angle)
    sum = np.sum(sv)
    print(f'{angle=},\tdegs_of_delay = {np.angle(sv, deg=True)},\tsum={np.abs(sum):.2f} phase={ np.angle(sum, deg=True) }')

f0 = 8166.666666666666
angle=0,	degs_of_delay = [0. 0. 0. 0.],	sum=4.00 phase=0.0
angle=30,	degs_of_delay = [   0. -180.    0. -180.],	sum=0.00 phase=-90.0
angle=45,	degs_of_delay = [   0.    105.44 -149.12  -43.68],	sum=0.65 phase=-21.837661840735517
angle=60,	degs_of_delay = [  0.    48.23  96.46 144.69],	sum=2.43 phase=72.34628195640316
angle=90,	degs_of_delay = [0. 0. 0. 0.],	sum=4.00 phase=2.1050127895604626e-14


In fact, we can plot the sensitivityv (and relative phase) as a function of angle in a radial plot:

In [4]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, polar=True)

def plot_sensitivity(f0, num_elements, distance_of_elements):
    # Calculate sensitivity for each angle
    angles = np.linspace(-90, 90, 181)  # Angles from -90 to 90 degrees

    sum_vectors = [np.sum(steering_vector(f0, angle, num_elements, distance_of_elements)) for angle in angles]
    sensitivities = np.abs(sum_vectors)
    phases = np.angle(sum_vectors) + PI # We add PI to keep the phase positive in the plot

    # we could convert to dB scale
    # dbnorm = lambda x: 20*np.log10(np.abs(x)/np.max(x));

    # Plot radial plot
    # fig, ax = plt.subplots(1, 1)
    ax.clear()
    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')
    ax.plot(np.radians(angles), sensitivities)
    ax.plot(np.radians(angles), phases, '-.')
    # plt.plot(np.radians(angles), sensitivities)
    ax.set_rlabel_position(0)
    # ax.set_xticks(np.array([-90, -45, 0, 45, 90])/180*PI)
    ax.set_thetalim(-1/2*PI, 1/2*PI)

    ax.set_xlabel('Angle (degrees)', fontsize=12, labelpad=15)
    ax.set_ylabel('Sensitivity', fontsize=12, labelpad=15)
    plt.title('Beamformer Sensitivity', fontsize=14, pad=20)
    plt.show()

default_f0 = speed_of_sound / 0.042 # wavelength equal to intermic distance (42mm)
@interact(f0 = IntSlider(min=100, max=Fs/2, step=1, value = default_f0),
          number_of_elements = IntSlider(min=1, max=32, step=1, value = 2),
          element_distance = IntSlider(min=10, max=150, step=1, value = 40)
         )
def plot(f0, number_of_elements, element_distance):
    plot_sensitivity(f0, number_of_elements, element_distance / 1000)


interactive(children=(IntSlider(value=8166, description='f0', max=22050, min=100), IntSlider(value=2, descript…